<a href="https://colab.research.google.com/github/ruvebenadie/PythonScipts/blob/main/NoZonesGarsfontein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import LineString, LinearRing, Point
from random import seed, random
import time



In [20]:

from google.colab import drive
drive.mount('/content/drive')



# Define file location
POINT_NAME = "/content/drive/MyDrive/Masters/Gars_Random.csv"
WARD_FILE = "/content/drive/MyDrive/Masters/garsfontein.shp"
NO_ZONES_FILE = "/content/drive/MyDrive/Masters/Parks.shp"

INPUT = "/content/drive/MyDrive/Masters/input_2053.gpkg"
OUTPUT = "/content/drive/MyDrive/Masters/output_2053.gpkg"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Define variables
min_dist = 67.73
max_dist = 95.78
buffer_dist = []

min_bearing = 0
max_bearing = 360
bearing = []

In [22]:
# seed random number generator
seed(1)

# Check if point is in no zones
def check_noZones(point):
    flag = True

    for index, row in nozones_df.iterrows():
        if (point.within(row.geometry)):
            flag = False

    return flag

# Function to determine the point within the min and max distance at a random bearing
def find_point(row):
    x = row.geometry.x
    y = row.geometry.y
    wardID = row.COMPNT_NM
    geom = wards_new['geometry'].loc[wards_new['COMPNT_NM'] == wardID].iat[0]
    i = 0

    while True:
        i =+ 1

        # Get random values
        buff_value = random()
        bearing_value = random()
        scaled_buff = min_dist + (buff_value * (max_dist - min_dist))
        scaled_bearing = min_bearing + (bearing_value * (max_bearing - min_bearing))

        # Algorithm for moving the point within the min and max buffer at a specified baering
        polygon = row.geometry.buffer(scaled_buff)

        east, south, west, north = polygon.bounds
        line_length = max(abs(east-west), abs(north-south)) * 2

        new_x = x + (np.sin(np.deg2rad(scaled_bearing)) * line_length)
        new_y = y + (np.cos(np.deg2rad(scaled_bearing)) * line_length)

        l = LineString([[x,y], [new_x, new_y]])
        lr = LinearRing(polygon.exterior.coords)
        intersections = lr.intersection(l)

        if (intersections.within(geom)) and check_noZones(intersections):
            return intersections.x, intersections.y
        elif (i == 500):
            return intersections.x, intersections.y

In [23]:
# Switch of copy df warning
pd.set_option('mode.chained_assignment', None)

# Load the Citizen CSV as a pandas dataframe, but only selected columns
crime_df = pd.read_csv(POINT_NAME, delimiter=",", low_memory=False)
print('Point layer successfully imported')


Point layer successfully imported


In [24]:
# Load the Wards SHP as a geopandas dataframe

wards_df = gpd.read_file(WARD_FILE)
wards_df.crs = {'init' :'epsg:4326'}
print('Polygon layer successfully imported')


Polygon layer successfully imported


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [25]:
# Load the no zones SHP as a geopandas dataframe
nozones_df = gpd.read_file(NO_ZONES_FILE)
nozones_df = nozones_df.to_crs({'init': 'epsg:2053'})
print('No zones layer successfully imported')

No zones layer successfully imported


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [26]:
# Convert the lat and long of the point layer to float
crime_df[['latitude', 'longitude']] = crime_df[['latitude', 'longitude']].astype(float)



In [27]:
# Convert the lat long to a point and add it into a geopandas dataframe
geometry = [Point(xy) for xy in zip(crime_df['longitude'], crime_df['latitude'])]
crs = {'init': 'epsg:4326'}
geo_df = GeoDataFrame(crime_df, crs=crs, geometry=geometry)
print('Point layer converted to spatial df')


Point layer converted to spatial df


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [28]:
# Merge the wards into a single polygon
cot = wards_df.geometry.unary_union

In [29]:
# Select all points within CoT
within_cot = geo_df[geo_df.geometry.within(cot)]
print(within_cot)

      latitude  longitude  area_in_me  confidence                    geometry  \
0   -25.810011  28.325399     17.7579      0.6749  POINT (28.32540 -25.81001)   
1   -25.781878  28.312204     28.2124      0.7497  POINT (28.31220 -25.78188)   
2   -25.778031  28.308398     52.7034      0.7559  POINT (28.30840 -25.77803)   
3   -25.761809  28.308801    253.1738      0.8345  POINT (28.30880 -25.76181)   
4   -25.762018  28.317482    255.2446      0.7888  POINT (28.31748 -25.76202)   
..         ...        ...         ...         ...                         ...   
302 -25.807710  28.307493     10.5362      0.6263  POINT (28.30749 -25.80771)   
303 -25.764565  28.278199     35.0083      0.7170  POINT (28.27820 -25.76456)   
304 -25.783017  28.324970     61.3267      0.7660  POINT (28.32497 -25.78302)   
305 -25.782360  28.317922     24.9887      0.6658  POINT (28.31792 -25.78236)   
306 -25.820758  28.286890     58.0256      0.7000  POINT (28.28689 -25.82076)   

        full_plus_  
0    5

In [30]:
# Save adjusted files to be
within_cot.to_file(INPUT, driver="GPKG")
print('Adjusted input files written to geopackage')

wards_new = wards_df.to_crs({'init': 'epsg:2053'})

# Adding polygon information to the points layer
merging = gpd.sjoin(within_cot, wards_df, how="left", op="within")

# Create a copy of the within_cot dataframe and change projection to UTM 35S WGS84 - units:meters
moved_point = within_cot.copy()
moved_point = moved_point.to_crs({'init': 'epsg:2053'})
merging = merging.to_crs({'init': 'epsg:2053'})

# Call the find_point function
moved_point['geometry'] = [Point(xy) for xy in zip(merging.apply(find_point, axis=1))]

# Change projection to WGS84 (SRID 4326)
moved_point = moved_point.to_crs({'init': 'epsg:4326'})

# Final results written out
moved_point.to_file(OUTPUT, driver="GPKG")
print('Output file is out into output geopackage')

Adjusted input files written to geopackage


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string 

Output file is out into output geopackage
